#### Competición kaggle:  
https://www.kaggle.com/competitions/reconocimiento-de-expresiones-faciales

## Librerias

In [21]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

## Data

In [22]:
# Rutas de los archivos CSV
train_csv_path = 'train_set.csv'
test_csv_path = 'test_set.csv'
sample_submission_path = 'sample_submision.csv'

# Leer los archivos CSV
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
sample_submission_df = pd.read_csv(sample_submission_path)

In [23]:
# Visualizar los primeros registros de los DataFrames
print(train_df.head())
print("-----------")
print(test_df.head())
print("-----------")
print(sample_submission_df.head())

   id_img                                  path  label
0       0      ../data/images/train/angry\0.jpg  angry
1       1      ../data/images/train/angry\1.jpg  angry
2      10     ../data/images/train/angry\10.jpg  angry
3   10002  ../data/images/train/angry\10002.jpg  angry
4   10016  ../data/images/train/angry\10016.jpg  angry
-----------
   id_img
0   10052
1   10065
2   10079
3   10095
4   10121
-----------
   id_img     label
0   10052  surprise
1   10065  surprise
2   10079  surprise
3   10095  surprise
4   10121  surprise


## Función para cargar y prepocesar las imagenes

In [4]:
# Definir rutas
ROOT_PATH = './data/images/'
TRAIN_PATH = ROOT_PATH + "train/"
TEST_PATH = ROOT_PATH + "test/"

In [24]:
# Función para cargar y preprocesar las imágenes de entrenamiento
def read_train_data(dataframe, reshape_dim=(32, 32)):
    X = []
    y = []
    for index, row in dataframe.iterrows():
        img_path = os.path.join(ROOT_PATH, row['path'].replace('../data/images/', ''))
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Leer en escala de grises
        if image is not None:
            image = cv2.resize(image, reshape_dim)  # Redimensionar a 32x32
            X.append(image)
            y.append(row['label'])
    
    X = np.array(X).reshape(-1, reshape_dim[0], reshape_dim[1], 1)  # Agregar canal de color
    y = np.array(y)
    return X, y


In [25]:
# Función para cargar y preprocesar las imágenes de test
def read_test_data(dataframe, reshape_dim=(32, 32)):
    X = []
    for index, row in dataframe.iterrows():
        img_path = os.path.join(TEST_PATH, f"{row['id_img']}.jpg")
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Leer en escala de grises
        if image is not None:
            image = cv2.resize(image, reshape_dim)  # Redimensionar a 32x32
            X.append(image)
    
    X = np.array(X).reshape(-1, reshape_dim[0], reshape_dim[1], 1)  # Agregar canal de color
    return X

In [26]:
# Cargar datos de entrenamiento y prueba
X_train, y_train = read_train_data(train_df)
X_test = read_test_data(test_df)

In [27]:
# Normalizar los datos
X_train = X_train / 255.0
X_test = X_test / 255.0

In [29]:
# Verificar las formas de los datos
print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_test:", X_test.shape)

Forma de X_train: (28821, 32, 32, 1)
Forma de y_train: (28821,)
Forma de X_test: (7066, 32, 32, 1)


### Ajustar etiquetas a números


In [30]:
# Como las etiquetas de las emociones son textos, necesitamos convertirlas a números

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

# Verificar las etiquetas transformadas
print("Etiquetas transformadas:", np.unique(y_train))

Etiquetas transformadas: [0 1 2 3 4 5 6]


-----------------------------------------------------------------------------------------------------------------

## Contruccion del modelo

### Usaremos una CNN (red neuronal convolucional)

In [31]:
# Configurar EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [32]:
# Definir el modelo con BatchNormalization y ajustes
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')  # Número de emociones
])

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
# Resumen del modelo
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 227,719 (889.53 KB)

 Trainable params: 226,759 (885.78 KB)

 Non-trainable params: 960 (3.75 KB)

### Entrenamiento del modelo

In [35]:
# Dividir el conjunto de datos para validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=42)

In [36]:
# Verificar las formas de los datos
print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_val:", X_val.shape)
print("Forma de y_val:", y_val.shape)

Forma de X_train: (23056, 32, 32, 1)
Forma de y_train: (23056,)
Forma de X_val: (5765, 32, 32, 1)
Forma de y_val: (5765,)


In [37]:
# Entrenar el modelo con EarlyStopping
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 33s 79ms/step - accuracy: 0.2175 - loss: 2.4602 - val_accuracy: 0.1795 - val_loss: 1.9074
Epoch 2/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 25s 69ms/step - accuracy: 0.3407 - loss: 1.7444 - val_accuracy: 0.4088 - val_loss: 1.5561
Epoch 3/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 25s 70ms/step - accuracy: 0.4024 - loss: 1.5573 - val_accuracy: 0.4243 - val_loss: 1.5150
Epoch 4/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 26s 73ms/step - accuracy: 0.4329 - loss: 1.4749 - val_accuracy: 0.3957 - val_loss: 1.5524
Epoch 5/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 27s 75ms/step - accuracy: 0.4461 - loss: 1.4270 - val_accuracy: 0.4749 - val_loss: 1.3652
Epoch 6/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 27s 74ms/step - accuracy: 0.4728 - loss: 1.3702 - val_accuracy: 0.4959 - val_loss: 1.3219
Epoch 7/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 39s 68ms/step - accuracy: 0.4871 - loss: 1.3388 - val_accuracy: 0.4323 - val_loss: 1.4620
Epoch 8/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 25s 68ms/step - accuracy: 0.4981 - loss: 1.3201 - 

-----------------------------------------------------------------

### Predicciones

In [38]:
# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)

221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [41]:
# Crear el DataFrame para submission
submission_df = pd.DataFrame({
    'id_img': test_df['id_img'],
    'label': label_encoder.inverse_transform(predictions)
})

In [42]:
# Guardar el archivo de submission
submission_df.to_csv('submission.csv', index=False)